In [165]:
import numpy as np
import os

In [166]:
%pwd

'C:\\Users\\llaur\\OneDrive\\Desktop\\turb\\proj-turbulence\\test'

In [167]:
%cd ../../../../../Downloads/ERA_wind

C:\Users\llaur\Downloads\ERA_wind


In [168]:
fine = np.load("ERA_wind.npz")

# Load

## Train

In [169]:
%cd ../../OneDrive/Desktop/turb/proj-turbulence/train

C:\Users\llaur\OneDrive\Desktop\turb\proj-turbulence\train


In [170]:
u = fine.get('wind_u')
v = fine.get('wind_v')

In [171]:
# for lat in range(121)
#     for lon in range(201):
#         # the set of wind speeds at all time stamps at this one point
#         wind_speeds = [np.sqrt((u[i][1][lat][lon])**2 + (v[i][1][lat][lon])**2) for i in range (744)]

List comprehension of wind speeds at level 225

In [172]:
wind_speeds = [[[np.sqrt((u[i][1][lat][lon])**2 + (v[i][1][lat][lon])**2) for lon in range(200)] for lat in range(120)] for i in range(744)]

In [174]:
len(wind_speeds), len(wind_speeds[0]), len(wind_speeds[0][0])

(744, 120, 200)

In [182]:
files = os.listdir(os.getcwd())

Add to coarse and fine lists in order

In [184]:
coarse_train = []
fine_train = []
for file in files: # loop thru files
    if file[0]=="w":
        timestamp = file[19:-4]
        coarse_train.append(np.load(file)) # load file, save into coarse
        fine_train.append(wind_speeds[int(timestamp)]) # load fine version

In [185]:
import json
with open("wind_speeds.json", "w") as file:
    json.dump(wind_speeds, file)

KeyboardInterrupt: 

595 samples, # lat points, # lon points

In [186]:
len(coarse_train), len(coarse_train[0]), len(coarse_train[0][0])

(595, 15, 25)

In [187]:
len(fine_train), len(fine_train[0]), len(fine_train[0][0])

(595, 120, 200)

## Val

In [188]:
%cd ../val

C:\Users\llaur\OneDrive\Desktop\turb\proj-turbulence\val


In [189]:
files = os.listdir(os.getcwd())
coarse_val = []
fine_val = []
for file in files: # loop thru files
    if file[0]=="w":
        timestamp = file[19:-4]
        coarse_val.append(np.load(file)) # load file, save into coarse
        fine_val.append(wind_speeds[int(timestamp)]) # load fine version

In [190]:
len(coarse_val), len(coarse_val[0]), len(coarse_val[0][0])

(75, 15, 25)

In [191]:
len(fine_val), len(fine_val[0]), len(fine_val[0][0])

(75, 120, 200)

## Test

In [192]:
%cd ../test

C:\Users\llaur\OneDrive\Desktop\turb\proj-turbulence\test


In [193]:
files = os.listdir(os.getcwd())
coarse_test = []
fine_test = []
for file in files: # loop thru files
    if file[0]=="w":
        timestamp = file[19:-4]
        coarse_test.append(np.load(file)) # load file, save into coarse
        fine_test.append(wind_speeds[int(timestamp)]) # load fine version

In [194]:
len(coarse_test), len(coarse_test[0]), len(coarse_test[0][0])

(74, 15, 25)

In [195]:
len(fine_test), len(fine_test[0]), len(fine_test[0][0]), type(fine_test[0][0][0])

(74, 120, 200, numpy.float64)

# CNN

In [196]:
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim

## Load Data

In [210]:
class PairedImageDataset(Dataset):
    def __init__(self, coarse_images, high_res_images, transform=None):
        self.coarse_images = coarse_images
        self.high_res_images = high_res_images
        self.transform = transform

    def __len__(self):
        return len(self.coarse_images)

    def __getitem__(self, idx):
        coarse_img = self.coarse_images[idx]
        high_res_img = self.high_res_images[idx]

        if self.transform:
            coarse_img = self.transform(coarse_img)
            high_res_img = self.transform(high_res_img)

        return coarse_img, high_res_img

transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
])

train = PairedImageDataset(np.array(coarse_train), np.array(fine_train), transform=transform)
test = PairedImageDataset(np.array(coarse_test), np.array(fine_test), transform=transform)
val = PairedImageDataset(np.array(coarse_val), np.array(fine_val), transform=transform)

# load
batch_size = 4
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val, batch_size=batch_size, shuffle=True)

In [211]:
np.array(coarse_train).shape, np.array(fine_train).shape

((595, 15, 25), (595, 120, 200))

## Define CNN

In [212]:
class CNN(nn.Module):
    def __init__(self):
        upscale = 8
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, upscale**2, kernel_size=3, padding=1),
            nn.ReLU()
        )
        self.depth_to_space = nn.PixelShuffle(upscale)
        
    def forward(self, x):
        x = self.conv_layers(x.float())
        x = self.depth_to_space(x)
        return x
    
model = CNN()

## Train CNN

In [213]:
import torch

In [214]:
criterion = nn.MSELoss()
optimizer = optim.NAdam(cnn.parameters(), lr=0.00005)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        
#         inputs = inputs.squeeze()
        targets = targets.squeeze() 
#         print("shapes::::", inputs.shape, targets.shape)
        outputs = model(inputs)
        
        loss = criterion(outputs, targets.squeeze().float())
        loss.backward()
        optimizer.step()
    # Validation
    model.eval()
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            val_loss = criterion(outputs, targets.squeeze())
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}')


C:\Users\llaur\miniconda3\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([4, 120, 200])) that is different to the input size (torch.Size([4, 1, 120, 200])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/10, Loss: 449.363037109375, Val Loss: 588.4797041252593
Epoch 2/10, Loss: 395.72430419921875, Val Loss: 560.7319868372161
Epoch 3/10, Loss: 454.10247802734375, Val Loss: 365.87505740383756
Epoch 4/10, Loss: 464.1881103515625, Val Loss: 463.16209173852405
Epoch 5/10, Loss: 429.2926025390625, Val Loss: 450.87124734734226
Epoch 6/10, Loss: 415.2659606933594, Val Loss: 410.6549552528233
Epoch 7/10, Loss: 451.0671081542969, Val Loss: 387.6818406630731
Epoch 8/10, Loss: 576.3726806640625, Val Loss: 331.99090897406484
Epoch 9/10, Loss: 588.3734130859375, Val Loss: 393.5991050704843
Epoch 10/10, Loss: 454.97222900390625, Val Loss: 406.9644133950446
